## Homework Assignment

A 2500 MWth PWR unit is to be operated in an equilibrium reload cycle fashion employing an 18-month refueling cycle with an anticipated 90% capacity factor. Use the following data to determine:

a) Beginning-of-cycle (BOC) core average reactivity required  
b) Number of burnable poison (BP) rods, if any, required

Assume total core loading of 65 MTU and no control rods are inserted at full power.

- The rate of change of core reactivity with core burnup (BU) without BP rods is  
  \begin{equation*}
  \frac{d\rho}{dBU} = -0.0012 \, \frac{\% \Delta \rho}{\text{MWD/T}}
  \end{equation*}

- BP reactivity worth at BOC: $ \rho_{BP} = 0.004\% \Delta \rho $ per BP rod
- Rate of change of BP reactivity with core BU  
  \begin{equation*}
  \frac{d\rho}{dBU} = -0.000085 \rho_{BP} \, \frac{\% \Delta \rho}{\text{MWD/T}}
  \end{equation*}

- Soluble boron reactivity worth = 0.01% $ \Delta \rho $ / ppm
- Moderator temperature coefficient (MTC) = 0 at 1800 ppm soluble boron


In [ ]:
from IPython.display import display, Math
import math

# Defining the constants and parameters
thermal_power = 2500  # in MWth
refueling_cycle = 18  # in months
capacity_factor = 90  # in percent
core_loading = 65  # in MTU
d_rho_d_burnup_no_bp = -0.0012  # in %Δρ per MWD/T without BP rods
bp_reactivity_worth = 0.004  # %Δρ per BP rod at BOC
d_rho_d_burnup_bp = -0.000085  # %Δρ per MWD/T per BP rod
boron_reactivity_worth = 0.01  # %Δρ per ppm soluble boron
mtc_at_1800_ppm = 0  # Moderator temperature coefficient at 1800 ppm

# Placeholder function to calculate BOC core average reactivity
def calculate_boc_reactivity(core_loading, d_rho_d_burnup_no_bp, burnup_limit):
    # Placeholder for reactivity calculation
    # Implement based on specific requirements
    
    return boc_reactivity

# Placeholder function to determine the number of BP rods needed
def calculate_bp_rods(boc_reactivity, bp_reactivity_worth):
    # Placeholder for BP rod calculation
    # Implement based on specific requirements
    
    return num_bp_rods

# Calculate BOC reactivity and BP rods required
boc_reactivity = calculate_boc_reactivity(core_loading, d_rho_d_burnup_no_bp, refueling_cycle)
num_bp_rods = calculate_bp_rods(boc_reactivity, bp_reactivity_worth)

print(f"BOC Core Average Reactivity Required: {boc_reactivity} %Δρ")
print(f"Number of BP rods required: {num_bp_rods}")
